# (Analysis will go here)
- note 1
- note 2
- note 3

In [39]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# Load csv files
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read csv files
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the datasets
full_school_data_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
full_school_data_df.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


# District Summary:

In [40]:
# Total number of unique schools:
school_count = len(full_school_data_df['school_name'].unique())
print(school_count)

15


In [41]:
# Total number of students:
# note: students may share same first/last name, so unique is not used here
student_count = full_school_data_df['student_name'].count()
print(student_count)

39170


In [42]:
# Total budget:
# source function [1]  https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html
budget_summ_df = full_school_data_df.pivot_table(index= 'school_name', values= 'budget')
total_budget = budget_summ_df['budget'].sum()
print(total_budget)

24649428


In [43]:
# Average math score (mean)
student_summ_df = full_school_data_df.pivot_table(index= 'Student ID')
avg_math_score = student_summ_df['math_score'].mean()
print(avg_math_score)

78.98537145774827


In [44]:
# Average readimg score (mean)
student_summ_df = full_school_data_df.pivot_table(index= 'Student ID')
avg_reading_score = student_summ_df['reading_score'].mean()
print(avg_reading_score)

81.87784018381414


In [45]:
# Percent passing math scores
passing_math_count = full_school_data_df[(full_school_data_df['math_score'] >= 70)].count()['student_name']
print(str(passing_math_count) + ' students passed math.')

percent_passing_math = passing_math_count/float(student_count)*100
print(str(percent_passing_math) + '% students passed math.')

29370 students passed math.
74.9808526933878% students passed math.


In [46]:
# Percent passing reading scores
passing_reading_count = full_school_data_df[(full_school_data_df['reading_score'] >= 70)].count()['student_name']
print(str(passing_reading_count) + ' students passed reading.')

percent_passing_reading = passing_reading_count/float(student_count)*100
print(str(percent_passing_reading) + '% students passed reading.')

33610 students passed reading.
85.80546336482001% students passed reading.


In [47]:
# Calculate overall passing percent, combine both math and reading here:
passing_both_count = full_school_data_df[(full_school_data_df['reading_score'] >= 70) & 
                                         (full_school_data_df['math_score'] >= 70)
                                                            ].count()['student_name']
print(str(passing_both_count) + ' students pass both math and reading.')

overall_passing_rate = passing_both_count / float(student_count)*100
print(str(overall_passing_rate) + '% students passed both math and reading.')

25528 students pass both math and reading.
65.17232575950983% students passed both math and reading.


# District Summary Table

In [48]:
# Create a high-level snapshot of these summary statistics as a DataFrame.
# source [0]: edX Boot Camps LLC. Module 4 Challenge.
summary_dict = {'Total Schools': school_count, 'Total Students': student_count, 'Total Budget': total_budget, 
                'Average Math Score': avg_math_score, 'Average Reading Score': avg_reading_score,
                '% Passing Math': percent_passing_math, '% Passing Reading': percent_passing_reading, 
                'Overall Passing': overall_passing_rate}

district_summ_df = pd.DataFrame([summary_dict])

district_summ_df["Total Students"] = district_summ_df["Total Students"].map("{:,}".format)
district_summ_df["Total Budget"] = district_summ_df["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summ_df
# print(district_summ_df)

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


# School Summary:

In [162]:
school_types = school_data.set_index(['school_name'])['type']
school_types_df = pd.DataFrame(school_types).rename(columns= {'type': 'School Type'})
print(school_types_df)

                      School Type
school_name                      
Huang High School        District
Figueroa High School     District
Shelton High School       Charter
Hernandez High School    District
Griffin High School       Charter
Wilson High School        Charter
Cabrera High School       Charter
Bailey High School       District
Holden High School        Charter
Pena High School          Charter
Wright High School        Charter
Rodriguez High School    District
Johnson High School      District
Ford High School         District
Thomas High School        Charter


In [165]:
per_school_stu_counts = full_school_data_df.groupby(['school_name']).count()['Student ID']
per_school_stu_counts_df = pd.DataFrame(per_school_counts).rename(columns= {'Student ID': 'Total Students'})
per_school_stu_counts_df

,Total Students
school_name,
Bailey High School,4976
Cabrera High School,1858
Figueroa High School,2949
Ford High School,2739
Griffin High School,1468
Hernandez High School,4635
Holden High School,427
Huang High School,2917
Johnson High School,4761


In [181]:
per_school_budget = full_school_data_df.groupby(['school_name']).mean()['budget']
per_school_budget_df = pd.DataFrame(per_school_budget).rename(columns= {'budget': 'Total School Budget'})
print(per_school_budget_df)
print(type(per_school_budget_df))

budget_per_student = per_school_budget/per_school_counts
budget_per_student_df = pd.DataFrame(budget_per_student).rename(columns= {0: 'Per Student Budget'})
print(budget_per_student_df)
print(type(budget_per_student_df))

                       Total School Budget
school_name                               
Bailey High School               3124928.0
Cabrera High School              1081356.0
Figueroa High School             1884411.0
Ford High School                 1763916.0
Griffin High School               917500.0
Hernandez High School            3022020.0
Holden High School                248087.0
Huang High School                1910635.0
Johnson High School              3094650.0
Pena High School                  585858.0
Rodriguez High School            2547363.0
Shelton High School              1056600.0
Thomas High School               1043130.0
Wilson High School               1319574.0
Wright High School               1049400.0
<class 'pandas.core.frame.DataFrame'>
                       Per Student Budget
school_name                              
Bailey High School                  628.0
Cabrera High School                 582.0
Figueroa High School                639.0
Ford High School     

In [187]:
per_school_math_scores = full_school_data_df.groupby(['school_name']).mean()['math_score']
per_school_math_scores_df = pd.DataFrame(per_school_math_scores).rename(columns= {'math_score': 'Average Math Score'})
print(per_school_math_scores_df)

per_school_reading_scores = full_school_data_df.groupby(['school_name']).mean()['reading_score']
per_school_reading_scores_df = pd.DataFrame(per_school_reading_scores).rename(columns= {'reading_score': 'Average Reading Score'})
print(per_school_reading_scores_df)

                       Average Math Score
school_name                              
Bailey High School              77.048432
Cabrera High School             83.061895
Figueroa High School            76.711767
Ford High School                77.102592
Griffin High School             83.351499
Hernandez High School           77.289752
Holden High School              83.803279
Huang High School               76.629414
Johnson High School             77.072464
Pena High School                83.839917
Rodriguez High School           76.842711
Shelton High School             83.359455
Thomas High School              83.418349
Wilson High School              83.274201
Wright High School              83.682222
                       Average Reading Score
school_name                                 
Bailey High School                 81.033963
Cabrera High School                83.975780
Figueroa High School               81.158020
Ford High School                   80.746258
Griffin High Sch

In [207]:
#TODO: calc percent passing math per school
# First, get the number of students per school with math scores of 70 or higher
# print(full_school_data_df)
students_passing_math = full_school_data_df.loc[(full_school_data_df['math_score'] >= 70)]
students_passing_math = (students_passing_math.groupby('school_name').count()['Student ID'] / full_school_data_df.groupby(['school_name']).count()['Student ID'])*100

students_passing_math_df = pd.DataFrame(students_passing_math).rename(columns= {'Student ID': '% Passing Math'})


print(students_passing_math_df)
print(type(students_passing_math_df))


#TODO: calc percent passing reading per school. Cast as DataFrame obj:
students_passing_reading = full_school_data_df.loc[(full_school_data_df['reading_score'] >= 70)]
students_passing_reading = (students_passing_reading.groupby('school_name').count()['Student ID'] / full_school_data_df.groupby(['school_name']).count()['Student ID'])*100

students_passing_reading_df = pd.DataFrame(students_passing_reading).rename(columns= {'Student ID': '% Passing Reading'})

print(students_passing_reading_df)
print(type(students_passing_reading_df))

                       % Passing Math
school_name                          
Bailey High School          66.680064
Cabrera High School         94.133477
Figueroa High School        65.988471
Ford High School            68.309602
Griffin High School         93.392371
Hernandez High School       66.752967
Holden High School          92.505855
Huang High School           65.683922
Johnson High School         66.057551
Pena High School            94.594595
Rodriguez High School       66.366592
Shelton High School         93.867121
Thomas High School          93.272171
Wilson High School          93.867718
Wright High School          93.333333
<class 'pandas.core.frame.DataFrame'>
                       % Passing Reading
school_name                             
Bailey High School             81.933280
Cabrera High School            97.039828
Figueroa High School           80.739234
Ford High School               79.299014
Griffin High School            97.138965
Hernandez High School        

In [214]:
students_passing_both = full_school_data_df.loc[(full_school_data_df['math_score'] >= 70) & (full_school_data_df['reading_score'] >= 70), :]
students_passing_both = (students_passing_both.groupby('school_name').count()['Student ID'] / full_school_data_df.groupby(['school_name']).count()['Student ID'])*100

students_passing_both_df = pd.DataFrame(students_passing_both).rename(columns= {'Student ID': '% Overall Passing'})

print(students_passing_both_df)
print(type(students_passing_both_df))




                       % Overall Passing
school_name                             
Bailey High School             54.642283
Cabrera High School            91.334769
Figueroa High School           53.204476
Ford High School               54.289887
Griffin High School            90.599455
Hernandez High School          53.527508
Holden High School             89.227166
Huang High School              53.513884
Johnson High School            53.539172
Pena High School               90.540541
Rodriguez High School          52.988247
Shelton High School            89.892107
Thomas High School             90.948012
Wilson High School             90.582567
Wright High School             90.333333
<class 'pandas.core.frame.DataFrame'>


In [215]:
# Display the School Summary DataFrame. Try using pandas merge method:
per_school_summary = pd.concat([school_types_df, per_school_stu_counts_df, per_school_budget_df, budget_per_student_df,
                                per_school_math_scores_df, per_school_reading_scores_df, students_passing_math_df, students_passing_reading_df,
                               students_passing_both_df], axis = 1)
print(per_school_summary)
print(type(per_school_summary))


                      School Type  Total Students  Total School Budget  \
school_name                                                              
Huang High School        District            2917            1910635.0   
Figueroa High School     District            2949            1884411.0   
Shelton High School       Charter            1761            1056600.0   
Hernandez High School    District            4635            3022020.0   
Griffin High School       Charter            1468             917500.0   
Wilson High School        Charter            2283            1319574.0   
Cabrera High School       Charter            1858            1081356.0   
Bailey High School       District            4976            3124928.0   
Holden High School        Charter             427             248087.0   
Pena High School          Charter             962             585858.0   
Wright High School        Charter            1800            1049400.0   
Rodriguez High School    District     

# School Summary Full Table

In [216]:
# rename columns with .rename method where needed:
# Nevermind... I ended up creating waaay too many individual DataFrames with correct column names instead.

# format column values using regex:
# Source: [0]: edX Boot Camps LLC. Module 4 Challenge.
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

per_school_summary


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,93.867121,95.854628,89.892107
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166


## Highest-Performing Schools (by % Overall Passing)

In [225]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
# Source [2] "Sort Pandas DataFrames method"

top_schools = per_school_summary.sort_values(by= ['% Overall Passing'], ascending= False).head(5)
top_schools


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing

In [226]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values(by= ['% Overall Passing'], ascending= True).head(5)
bottom_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [ ]:
# Use the code provided to separate the data by grade
# Source [0] edX Boot Camps LLC. 
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

